<a href="https://colab.research.google.com/github/salvapineda/bilevel/blob/master/UnitCommitment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unit Commitment in Python

## Requirements

In [1]:
!pip install pyomo
import os
import pandas as pd
import numpy as np
import pyomo.environ as pe

     |████████████████████████████████| 9.5MB 6.7MB/s 
     |████████████████████████████████| 256kB 42.8MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 163kB 38.7MB/s 


## Input Data

In [2]:
print('==============================')
print('Unit data')
print('==============================')
data_gen = [[0,0.1,10,0,400],[1,0.1,20,0,400]]
ng = len(data_gen)
gen = pd.DataFrame(data_gen,index=['g'+str(i) for i in range(ng)],columns=['bus','a','b','min','max'])
print(gen)
print('==============================')
print('Line data')
print('==============================')
data_lin = [[0,1,1,1000],[0,2,2,70],[1,2,3,1000]]
nl = len(data_lin)
lin = pd.DataFrame(data_lin,index=['l'+str(i) for i in range(nl)],columns=['from','to','sus','cap'])
print(lin)
print('==============================')
print('Demand data')
print('==============================')
data_dem = [[49.4, 105.81, 0.0], [146.81, 159.56, 0.0], [104.2, 148.69, 0.0], [222.75, 134.48, 0.0], [61.84, 163.19, 0.0], [65.81, 102.29, 0.0], 
            [177.4, 160.33, 0.0], [9.66, 136.83, 0.0], [29.6, 141.61, 0.0], [186.57, 161.3, 0.0], [127.28, 115.28, 0.0], [248.79, 141.13, 0.0], 
            [46.01, 114.35, 0.0], [148.17, 117.66, 0.0], [131.89, 153.54, 0.0], [75.98, 165.17, 0.0], [184.68, 129.26, 0.0], [102.42, 156.23, 0.0],
            [89.28, 176.49, 0.0], [183.79, 148.23, 0.0], [111.95, 158.81, 0.0], [61.15, 102.98, 0.0], [174.65, 107.03, 0.0], [168.12, 124.58, 0.0]]
dem = pd.DataFrame(data_dem,index=['t'+str(i) for i in range(nt)],columns=['b'+str(i) for i in range(nb)])
print(dem)
print('==============================')
print('Renewable data')
print('==============================')
data_ren = [[0.0, 0.0, 39.07], [0.0, 0.0, 24.72], [0.0, 0.0, 22.42], [0.0, 0.0, 39.75], [0.0, 0.0, 43.6], [0.0, 0.0, 21.09], [0.0, 0.0, 37.59], 
            [0.0, 0.0, 41.89], [0.0, 0.0, 26.53], [0.0, 0.0, 41.02], [0.0, 0.0, 15.55], [0.0, 0.0, 7.09], [0.0, 0.0, 38.47], [0.0, 0.0, 29.11], 
            [0.0, 0.0, 9.49], [0.0, 0.0, 24.97], [0.0, 0.0, 34.4], [0.0, 0.0, 23.69], [0.0, 0.0, 26.87], [0.0, 0.0, 0.37], [0.0, 0.0, 11.89], 
            [0.0, 0.0, 34.84], [0.0, 0.0, 39.26], [0.0, 0.0, 33.43]]
nt = len(data_ren)
nb = len(data_ren[0])
ren = pd.DataFrame(data_ren,index=['t'+str(i) for i in range(nt)],columns=['b'+str(i) for i in range(nb)])
print(ren)

Unit data
    bus    a   b  min  max
g0    0  0.1  10    0  400
g1    1  0.1  20    0  400
Line data
    from  to  sus   cap
l0     0   1    1  1000
l1     0   2    2    70
l2     1   2    3  1000
Demand data


NameError: ignored

## Solving Unit Commitment

In [ ]:
# Model
m = pe.ConcreteModel()
# Sets
m.g = pe.Set(initialize=list(range(ng)),ordered=True)
m.l = pe.Set(initialize=list(range(nl)),ordered=True)
m.b = pe.Set(initialize=list(range(nb)),ordered=True)
m.t = pe.Set(initialize=list(range(nt)),ordered=True)
# Variables
m.z = pe.Var()
m.pro = pe.Var(m.g,m.t,within=pe.NonNegativeReals)
m.u = pe.Var(m.g,m.t,within=pe.Binary)
m.shd = pe.Var(m.b,m.t,within=pe.NonNegativeReals)
m.spl = pe.Var(m.b,m.t,within=pe.NonNegativeReals)
m.ang = pe.Var(m.b,m.t)
m.flw = pe.Var(m.l,m.t)
# Objective function
def obj_rule(m):
  return m.z
m.obj = pe.Objective(rule=obj_rule)
# Definition cost
def cost_def_rule(m):
  return m.z == sum(gen['a'][g]*m.pro[g,t]*m.pro[g,t] + gen['b'][g]*m.pro[g,t] for g in m.g for t in m.t) + sum(cs*m.shd[b,t] for b in m.b for t in m.t)
m.cost_def = pe.Constraint(rule=cost_def_rule)
# Energy balance
def bal_rule(m,b,t):
  return sum(m.pro[g,t] for g in m.g if gen['bus'][g] == b) + ren.iloc[t,b] + m.shd[b,t] + sum(m.flw[l,t] for l in m.l if lin['to'][l] == b) == dem.iloc[t,b] + m.spl[b,t] + sum(m.flw[l,t] for l in m.l if lin['from'][l] == b)
m.bal = pe.Constraint(m.b, m.t, rule=bal_rule)
# Minimum generation
def min_gen_rule(m,g,t):
  return m.pro[g,t] >= m.u[g,t]*gen['min'][g]
m.min_gen = pe.Constraint(m.g, m.t, rule=min_gen_rule)
# Maximum generation
def max_gen_rule(m,g,t):
  return m.pro[g,t] <= m.u[g,t]*gen['max'][g]
m.max_gen = pe.Constraint(m.g, m.t, rule=max_gen_rule)
# Maximum spilage
def max_spil_rule(m,b,t):
  return m.spl[b,t] <= ren.iloc[t,b]
m.max_spil = pe.Constraint(m.b, m.t, rule=max_spil_rule)
# Maximum shedding
def max_shed_rule(m,b,t):
  return m.shd[b,t] <= dem.iloc[t,b]
m.max_shed = pe.Constraint(m.b, m.t, rule=max_shed_rule)
# Power flow definition
def flow_rule(m,l,t):
  return m.flw[l,t] == lin['sus'][l]*(m.ang[lin['from'][l],t] - m.ang[lin['to'][l],t])
m.flow = pe.Constraint(m.l, m.t, rule=flow_rule)
# Max power flow
def max_flow_rule(m,l,t):
  return m.flw[l,t] <= lin['cap'][l]
m.max_flow = pe.Constraint(m.l, m.t, rule=max_flow_rule)
# Min power flow
def min_flow_rule(m,l,t):
  return m.flw[l,t] >= -lin['cap'][l]
m.min_flow = pe.Constraint(m.l, m.t, rule=min_flow_rule)
# We solve the optimization problem
os.environ['NEOS_EMAIL'] = 'xxx@gmail.com'
solver_manager = pe.SolverManagerFactory('neos')
opt = pe.SolverFactory('cplex')
opt.options['threads'] = 1
opt.options['mipgap'] = 1e-9
res = solver_manager.solve(m,opt=opt,symbolic_solver_labels=True,tee=True)
print(res['Solver'][0])

## Solution Output

In [ ]:
# Print output
print('==============================')
print('Unit commitment')
print('==============================')
u = pd.DataFrame([[round(m.u[g,t].value,2) for g in m.g] for t in m.t],index=['t'+str(i) for i in range(nt)],columns=['g'+str(i) for i in range(ng)])
print(u)
pro = pd.DataFrame([[round(m.pro[g,t].value,2) for g in m.g] for t in m.t],index=['t'+str(i) for i in range(nt)],columns=['g'+str(i) for i in range(ng)])
print('==============================')
print('Unit production')
print('==============================')
print(pro)
flw = pd.DataFrame([[round(m.flw[l,t].value,2) for l in m.l] for t in m.t],index=['t'+str(i) for i in range(nt)],columns=['l'+str(i) for i in range(nl)])
print('==============================')
print('Power flows')
print('==============================')
print(flw)
shd = pd.DataFrame([[round(m.shd[b,t].value,2) for b in m.b] for t in m.t],index=['t'+str(i) for i in range(nt)],columns=['l'+str(i) for i in range(nb)])
print('==============================')
print('Load shedding')
print('==============================')
print(shd)
spl = pd.DataFrame([[round(m.spl[l,t].value,2) for b in m.b] for t in m.t],index=['t'+str(i) for i in range(nt)],columns=['l'+str(i) for i in range(nb)])
print('==============================')
print('Wind spillage')
print('==============================')
print(spl)